# Titanic Dataset - Classificazione

In questo notebook utilizzeremo il dataset `Titanic` che contiene alcune informazioni relative ai passeggeri del Titanic, nave tristemente famosa in quanto è affondata durante il viaggio di inaugurazione causando centinaia di morti. 

Utilizzeremo questo dataset per sviluppare un modello di **classificazione**, che ci permetterà di prevedere la sopravvivenza o meno dei passeggeri. Nella prima sezione vedremo come implementare un Albero decisionale per la classificazione. Nella seconda vedremo altri modelli di classificazione un po' più complessi.

**Nota**: scorrendo il notebook noterai che ci sono alcune celle di codice che contengono puntini (...), quelle sono le parti che dovrai completare aiutandoti con le esercitazioni che abbiamo svolto durante le lezioni e con le presentazioni che avete seguito. Altre celle sono invece già completate e dovrai solo cliccare 'run' per studiarne l'output. Troverai anche alcune domande che ti guideranno nella descrizione e nell'analisi che svolgerai per il report finale.

<a id="0"></a> <br>

# Indice
1. [Pre-processing dei dati](#1)
2. [Exploratory Data Analysis](#2)
3. [Implementazione e valutazione del modello Decision Tree](#3)
4. [Altri modelli di classificazione](#4)

<a id="1"></a> <br>
## 1. Pre-processing dei dati

Lo step fondamentale prima di applicare un modello di Machine Learning è quello di studiare le caratteristiche principali dei dati per renderli utilizzabili dal modello che sceglieremo. In questa sezione, metteremo in pratica le tecniche viste nelle precedenti lezioni per analizzare il dataset, in particolare dovremo:
- caricare il dataset
- estrarre le prime descrizioni generali (dimensione, tipo di dati, variabili, ...)
- gestire i valori mancanti
- gestire variabili categoriche

- Caricamento del dataset e descrizione generale

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Caricamento del dataset
df = pd.read_csv('../data/Titanic_data.csv')

# Stampa le prime dieci righe
df.head(10)

In [ ]:
# Che dimensione ha il dataset? Quante righe e quante colonne ha?
df.shape

In [ ]:
# Quali sono i nomi delle variabili presenti nel dataset? I nomi delle variabili sono contenuti nella lista delle colonne del dataframe
df.columns.to_list()

Il dataset contiene 8 variabili:

1. **PassengerId**: ID del passeggero

2. **Pclass**: classe del biglietto del passeggero passeggero (1 = prima classe, 2 = seconda classe, 3 = terza classe)

3. **Name**: nome del passeggero

4. **Sex**: sesso del passeggero

5. **Age**:: età del passeggero

6. **SibSp**: indica quanti fratelli/sorelle o mogli/spose il passeggero aveva sul Titanic

7. **Parch**: indica quanti mamme/papà o figli il passeggero aveva sul Titanic

8. **Ticket**: numero diel biglietto

9. **Fare**: tariffa pagata dal passeggero

10. **Cabin**: numero della cabina

11. **Embarked**: porto d'imbarco del passegero (C = Cherbourg, Q = Queenstown, S = Southampton)

12. **Survived**: indica se il passeggero è sopravvissuto o no (0 = No, 1 = Yes)

In [ ]:
# Alcune informazioni importanti sul dataset (possiamo usare il metodo .info())
df.info()

In [ ]:
# Tipo di dato in ogni colonna (possiamo usare il metodo .dtypes)
df.dtypes

In [ ]:
# Caratteristiche statistiche principali per le variabili numeriche
df.describe()

- Valori mancanti

In [ ]:
# Quanti valori nulli ci sono in ogni colonna?
df.isna().sum()

In [ ]:
# Calcoliamo la percentuale di dati mancanti per ogni variabile
missing_values = df.isna().sum()
missing_percentage = (missing_values / len(df)) * 100
print("Missing Values and Percentages:\n", pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
}))

In [ ]:
# Come gestiamo i valori nulli?
# Ci sono vari metodi per gestire i dati mancanti: possiamo eliminare dal dataset le righe corrispondenti, sostituirli con un valore medio o con il valore mediano, ...
# In questo caso, ci sono tre variabili che presentano dei valorinulli: Age, Cabin, Embarked.

# Per quanto riguarda la variabile "Cabin", siccome oltre il 70% dei passeggeri presenti nel dataset hanno un valore nullo e
# tale variabile non sembra essere correlata con la variabile survived, decidiamo di eliminare tutta la variabile dal dataset
df.drop('Cabin', axis=1, inplace=True)

In [ ]:
# Per quanto riguarda la variabile "Age" siccome c'è un numero considerevole di dati mancanti (quasi il 20%), 
# proviamo a sostituire tali valori mancanti con la mediana che è più robusta agli outlier rispetto alla media)
df['Age'] = df['Age'].fillna(df['Age'].median())

In [ ]:
# Infine, per quanto riguarda la variabile 'Embarked' siccome ci sono solo due valori mancanti li sostituiamo con la moda (valore più frequente)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

- Variabili categoriche

Ci sono tre variabili categoriche:
- **Pclass**: Ordinale (1st, 2nd, 3rd class) -> non necessita di encoding in quanto i valori sono già numerici (1,2,3)
- **Sex**: Nominale (Male, Female) -> Label Encoding (Male -> 0, Female-> 1)
- **Embarked**: Nominale (C, Q, S) -> One-Hot encoding

In [ ]:
# Label Encoding per 'Sex' (binaria -> 'male': 0, 'female': 1)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1}) # Inserisci i due numeri in cui vengono trasformate le classi 'male' e 'female'

In [ ]:
# One-Hot Encoding per 'Embarked'
from sklearn.preprocessing import OneHotEncoder

# Inizializzazione dell'encoder
encoder = OneHotEncoder(sparse_output=False)

# Slezioniamo la variabile categorica da codificare
embarked_encoded = encoder.fit_transform(df[['Embarked']])

# Convertiamo i dati codificati in un DataFrame con nomi delle colonne significativi
embarked_encoded_df = pd.DataFrame(embarked_encoded, columns=encoder.get_feature_names_out(['Embarked']))

# Aggiungiamo le colonne codificate nel DataFrame originale
df = pd.concat([df, embarked_encoded_df], axis=1)

# Eliminiamo la colonna originale 'Embarked' dal dataset
df = df.drop('Embarked', axis=1)

# Mostriamo il DataFrame aggiornato
df.head()

<a id="2"></a> <br>
## 2. Exploratory Data Analysis

In questa sezione utilizzeremo dei metodi di visualizzazione dei dati per continuare ad analizzare le caratteristiche del dataset. 
In particolare dovremo:
- plottare la correlation heatmap per valutare la correlazione tra le diverse variabili
- studiare come sono distribuite le diverse variabili

In [ ]:
# Stampiamo la correlation heatmap per valutare la correlazione tra le variabili
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

- Distribuzione delle variabili

In [ ]:
# Com'è distribuito il numero di sopravvissuti in base al sesso?
df.groupby(['Sex', 'Survived'])['Survived'].count() # Usa groupby sulle colonne 'Sex' e 'Survived'

In [ ]:
# Visualizziamo il risultato precedente
sns.countplot(x='Sex',hue='Survived',data=df)
plt.show()

In [ ]:
# Com'è distribuito il numero di sopravvissuti in base al lla classe del biglietto?
df.groupby(['Pclass', 'Survived'])['Survived'].count() # Usa groupby sulle colonne 'Pclass' e 'Survived'

In [ ]:
# Visualizziamo il risultato precedente
sns.countplot(x='Pclass', hue='Survived', data=df)
plt.show()

In [ ]:
# Cambia i nomi delle variabili per visualizzare i diversi plot

# Bar plot
sns.countplot(x='Survived', data=df)
plt.xlabel('Survival Status')
plt.ylabel('Count')
plt.title('Survival Count')
plt.show()

# Histogram
plt.hist(df['Age'], bins=10)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Distribution of Age')
plt.show()

# Scatter plot
plt.scatter(df['Age'], df['Fare'])
plt.xlabel('Age')
plt.ylabel('Fare')
plt.title('Age vs. Fare')
plt.show()

# Box plot
sns.boxplot(x=df['Survived'], y=df['Fare'])
plt.xlabel('Survival Status')
plt.ylabel('Fare')
plt.title('Survival Status vs. Fare')
plt.show()

- Colonne da eliminare

In [ ]:
# Colonne da eliminare (non servono per il modello): 'PassengerId','Name','Ticket'
columns_to_drop = ["PassengerId","Name","Ticket"]

# Eliminiamo le colonne non necessarie al modello
df_cleaned = df.drop(columns=columns_to_drop)

<a id="3"></a> <br>

## 3. Implementazione e valutazione del modello di Decision Tree

In questa sezione costruiremo e alleneremo il Decision tree Classifier (seguendo gli step illustrati nella presentazione). Infine valuteremo il modello ottenuto calcolando la matrice di confusione e le metriche ad essa associate.

DecisionTreeClassifier (documentazione): https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Definiamo le variabili di input (X) e di output (y)
# La colonna da prevedere è quella che si riferisce alla sopravvivenza o meno del passeggero ('Survived')
X = df_cleaned.drop(columns=['...' ])
y= df_cleaned['...' ]

In [ ]:
# Dividiamo i dati in training (80%) e test(20%)
...

In [ ]:
# Controlliamo la dimensione del dataset di training e di test
...
...
...
...

In [ ]:
# Creiamo il Decision Tree Classifier
model = ... # Fissiamo il random_state (random_state=42)

In [ ]:
# Alleniamo il modello sui dati di training
...

In [ ]:
# Facciamo previsioni sui dati di test
y_predict = ...

In [ ]:
# Valutiamo il modello

# Stampiamo la matrice di confusione
confusion = confusion_matrix(y_test, y_predict)
print(confusion)

In [ ]:
# Accuratezza del modello
accuracy = ...
print(accuracy)

In [ ]:
# Report dettagliato delle metriche
report = ...
print(report)

In [ ]:
# Plot dell'albero
from sklearn.tree import plot_tree
plt.figure(figsize=(100,100))
plot_tree(model, filled=True, feature_names=X_train.columns, fontsize=10)
plt.show()

Il modello Decision Tree permette di settare i valori per diversi parametri, ma quali sono i valori migliori? 
**GridSearch** che è un metodo per trovare automaticamente i migliori parametri di un modello. Invece di provare i valori manualmente, **GridSearchCV** testa tutte le combinazioni possibili di parametri e sceglie quella che dà le migliori prestazioni, basandosi su una metrica di valutazione (es. R² o RMSE).

In [2]:
from sklearn.model_selection import GridSearchCV

In [ ]:
cross_valid_scores = {}

# Definiamo i valori possibili per la profondità massima dell'albero
parameters = {
    "max_depth": [3, 5, 7, 9, 11, 13],
}

model_desicion_tree = DecisionTreeClassifier(
    random_state=123,
    class_weight='balanced', # Bilancia automaticamente le classi se sono sbilanciate
)

model_desicion_tree = GridSearchCV(
    model_desicion_tree, # Modello da ottimizzare
    parameters, # Dizionario dei parametri da testare
    cv=5, # Numero di suddivisioni per la cross-validation (5-fold CV)
    scoring='accuracy', # Metrica di valutazione da ottimizzare
)

model_desicion_tree.fit(X_train, y_train)

print(f'Migliore combinazione di parametri trovati {model_desicion_tree.best_params_}')
print(
    f'Media delle accuratezze ottenuta con la cross-validation per il miglior modello: ' + \
    f'{model_desicion_tree.best_score_:.3f}'
)
cross_valid_scores['desicion_tree'] = model_desicion_tree.best_score_

<a id="4"></a> <br>

## 4. Altri modelli di classificazione

In questa sezione testremo dei modelli di classificazione alternativi al Decision Tree
- Random Forest Classifier
- Logistic Regression

*Random Forest Classifier*: È un modello di machine learning basato su più alberi decisionali. Ogni albero fa una previsione e la classe finale viene scelta con una votazione della maggioranza, ovvero la classe che ottiene il maggior numero di "voti" da parte degli alberi viene scelta come previsione del modello.

*Logistic Regression*: Nonostante il nome, è un modello di classificazione, non di regressione. Pprevede la probabilità che un'osservazione appartenga a una certa classe (es. 0 o 1). È semplice, veloce e adatto per problemi di classificazione binaria.

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Creiamo il modello di Random Forest Classifier
rf =...

# Alleniamo il modello sui dati di training
...

# Facciamo previsioni sui dati di test
rf_pred = ...

In [ ]:
# Valutiamo il modello

# Stampiamo la matrice di confusione
confusion = confusion_matrix(y_test, rf_pred)
print(confusion)

In [ ]:
# Accuratezza del modello
accuracy = ...
print(accuracy)

In [ ]:
# Report dettagliato delle metriche
report = ...
print(report)

- Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Creiamo il modello di LogisticRegression
logreg = ...

# Alleniamo il modello su dati di training
...

# Facciamo previsioni sui dati di test
logreg_pred = ...

In [ ]:
# Valutiamo il modello

# Stampiamo la matrice di confusione
confusion = confusion_matrix(y_test, logreg_pred)
print(confusion)

In [ ]:
# Accuratezza del modello
accuracy = ...
print(accuracy)

In [ ]:
# Report dettagliato delle metriche
report = ...
print(report)

## Domande

- Qual è il modello con l'accuratezza migliore?
- Cosa succederebbe se invece di dividere il dataset in training set e test in modo randomico prendessimo come training set solo i passeggeri maschi e come test set solo le passeggere femmine?
- Tra i parametri che possono essere scelti in un modello ad Albero Decisionale c'è la profondità dell'albero, ovvero quanti livelli (o nodi) dell'albero verranno considerati. Cosa potrebbe succedere se considerassimo un numero troppoo elevato di nodi? 